In [1]:
import pyspark

In [2]:
pyspark.__version__

'3.3.2'

In [4]:
pyspark.__file__

'/home/almaz.alpyssov/spark/spark-3.3.2-bin-hadoop3/python/pyspark/__init__.py'

In [52]:
from pyspark.sql import SparkSession
from pyspark.sql import types

In [6]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/09 06:42:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-07 12:45:06--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.187, 18.155.128.222, 18.155.128.46, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-07 12:45:06 (187 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [7]:
!head taxi_zone_lookup.csv

"LocationID","Borough","Zone","service_zone"
1,"EWR","Newark Airport","EWR"
2,"Queens","Jamaica Bay","Boro Zone"
3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
4,"Manhattan","Alphabet City","Yellow Zone"
5,"Staten Island","Arden Heights","Boro Zone"
6,"Staten Island","Arrochar/Fort Wadsworth","Boro Zone"
7,"Queens","Astoria","Boro Zone"
8,"Queens","Astoria Park","Boro Zone"
9,"Queens","Auburndale","Boro Zone"


In [8]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [9]:
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [20]:
df.write.parquet('zones')

In [92]:
parqFile = spark.read.parquet("zones")
parqFile.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [96]:
parqFile.createOrReplaceTempView("zones")

In [99]:
zones = spark.sql("select zone from zones where LocationId = 105")
zones.show()

+--------------------+
|                zone|
+--------------------+
|Governor's Island...|
+--------------------+



In [10]:
!ls -la

total 44
drwxrwxr-x  4 almaz.alpyssov almaz.alpyssov  4096 Mar  9 06:44 .
drwxr-xr-x 19 almaz.alpyssov almaz.alpyssov  4096 Mar  7 10:36 ..
drwxrwxr-x  2 almaz.alpyssov almaz.alpyssov  4096 Mar  9 06:42 .ipynb_checkpoints
-rw-rw-r--  1 almaz.alpyssov almaz.alpyssov  9174 Mar  9 06:44 hello_spark.ipynb
-rw-rw-r--  1 almaz.alpyssov almaz.alpyssov 12331 Feb 22  2024 taxi_zone_lookup.csv
drwxr-xr-x  2 almaz.alpyssov almaz.alpyssov  4096 Mar  7 12:48 zones


In [11]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2025-03-09 06:51:18--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.6, 18.155.128.187, 18.155.128.222, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294.61M  49.4MB/s    in 6.5s    

2025-03-09 06:51:25 (45.5 MB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [19]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet

--2025-03-09 06:57:15--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.187, 18.155.128.222, 18.155.128.46, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-01.parquet’

fhv_tripdata_2021-0 100%[===================>]  11.33M  14.7MB/s    in 0.8s    

2025-03-09 06:57:17 (14.7 MB/s) - ‘fhv_tripdata_2021-01.parquet’ saved [11886281/11886281]



In [13]:
parqFile = spark.read.parquet("fhvhv_tripdata_2021-01.parquet")

In [20]:
parqFile2 = spark.read.parquet("fhv_tripdata_2021-01.parquet")

In [14]:
parqFile.createOrReplaceTempView("parquetFile")

In [21]:
parqFile2.createOrReplaceTempView("parquetFile2")

In [22]:
lines = spark.sql("select * from parquetFile2 limit 100")

In [26]:
lines = spark.sql("select count(*) from parquetFile")

In [27]:
lines.show()

+--------+
|count(1)|
+--------+
|11908468|
+--------+



In [29]:
parqFile2.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [31]:
df = parqFile.repartition(24)

In [33]:
df.write.parquet('fhvhv/2021/01/')

AnalysisException: path file:/home/almaz.alpyssov/notebooks/fhvhv/2021/01 already exists.

In [36]:
parqFile.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [76]:
df = parqFile.select('hvfhs_license_num', 'dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003')

In [77]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-----------------+--------------------+-------------------+-------------------+------------+------------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|
|           HV0003|              B02764|2021-01-01 00:14:30|2021-01-01 00:50:27|          71|         226|
|           HV0003|              B028

In [78]:
from pyspark.sql import functions as F

In [79]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-----------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|pickup_date|dropoff_date|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-----------+------------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166| 2021-01-01|  2021-01-01|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167| 2021-01-01|  2021-01-01|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142| 2021-01-01|  2021-01-01|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143| 2021-01-01|  2021-01-01|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08

In [70]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [80]:
crazy_stuff('B02884')

's/b44'

In [87]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [88]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-----------------+--------------------+-------------------+-------------------+------------+------------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|
|           HV0003|              B02764|2021-01-01 00:14:30|2021-01-01 00:50:27|          71|         226|
|           HV0003|              B028

In [89]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .show()
#.withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \ # not working

+-----------------+--------------------+-------------------+-------------------+------------+------------+-----------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|pickup_date|dropoff_date|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-----------+------------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166| 2021-01-01|  2021-01-01|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167| 2021-01-01|  2021-01-01|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142| 2021-01-01|  2021-01-01|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143| 2021-01-01|  2021-01-01|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08